In [1]:
import os
import sys
import json
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
import numpy as np

In [2]:
with open("DO_record_per_line.json", encoding='utf-8') as file:
    data = file.readlines()
data = list(map(lambda x: json.loads(x),data))
data = pd.DataFrame(data)

In [3]:
def calc_cosine(a,b):
    if sum(a) == 0 or sum(b) == 0:
        return 0
    return sum(a*b)/np.sqrt(sum(a**2)*sum(b**2))

In [4]:
target_courses=[[21545, u'en', u"The Divine Comedy: Dante's Journey to Freedom, Part 2 (Purgatorio)"], [7802, u'en', u'AE1110x: Introduction to Aeronautical Engineering'], [26228, u'es', u'Como ser Excelente Alumno - Udemy'], [196, u'es', u'Egiptolog\xeda (Egyptology)'], [12917, u'ru', u'\u041f\u0440\u0438\u043d\u0446\u0438\u043f\u044b \u044d\u0444\u0444\u0435\u043a\u0442\u0438\u0432\u043d\u043e\u0433\u043e \u0443\u043f\u0440\u0430\u0432\u043b\u0435\u043d\u0438\u044f \u0438\u043d\u0432\u0435\u0441\u0442\u0438\u0446\u0438\u043e\u043d\u043d\u044b\u043c\u0438 \u043f\u0440\u043e\u0435\u043a\u0442\u0430\u043c\u0438'], [21319, u'ru', u'\u041a\u043e\u043c\u043f\u0435\u0442\u0435\u043d\u0442\u043d\u043e\u0441\u0442\u043d\u043e-\u043e\u0440\u0438\u0435\u043d\u0442\u0438\u0440\u043e\u0432\u0430\u043d\u043d\u043e\u0435 \u043e\u0431\u0443\u0447\u0435\u043d\u0438\u0435 \u0432 \u0432\u044b\u0441\u0448\u0435\u043c \u043e\u0431\u0440\u0430\u0437\u043e\u0432\u0430\u043d\u0438\u0438']]

In [5]:
target_df = pd.DataFrame(target_courses, columns=['id','lang','name'])

In [6]:
languages = target_df.lang.unique()

In [7]:
result = {}
for lang in languages:
    data_lang = data[data.lang == lang].reset_index()
    vectorizer = TfidfVectorizer(max_features=25000, stop_words=ENGLISH_STOP_WORDS)
    data_tfidf = vectorizer.fit_transform(data_lang.desc).toarray()
    for course_id in target_df[target_df.lang==lang].id:
        row_number = data_lang[data_lang.id==course_id].index[0]
        result_dict = {}
        for i in range(len(data_tfidf)):
            if i != row_number:
                result_dict[int(data_lang.iloc[i].id)] = calc_cosine(data_tfidf[row_number], data_tfidf[i])
        result_tup = [(key, result_dict[key]) for key in result_dict.keys()]
        result[course_id]= np.array(sorted(result_tup,key=lambda x:-x[1]))[:10,0]
            

In [8]:
for key in result.keys():
    result[key] = list(map(int, result[key]))

In [9]:
with open("lab07.json", 'w') as file:
    json.dump(result,file)